# Assignment 3

In [145]:
import numpy as np
import pandas as pd 
import scipy as sci
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from scipy.signal import correlate

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

In [80]:
# Import data
D = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/AllStations_temperature_h_2017.dat', 
                 sep='\s+', header=1, usecols=[0,35])

In [81]:
# Convert time in D from MatLab time to Python Time
D['Time'] = D['NaN'].apply(lambda matlab_datenum: 
                             dt.datetime.fromordinal(int(matlab_datenum)) 
                             + dt.timedelta(days=matlab_datenum%1)
                             - dt.timedelta(days = 366)) 

# Rename the columns
D2 = D.rename(index=str, 
                columns={"NaN": "MatLab Time", "48.4623": "Temperature"})

In [82]:
# Reorder columns 
cols = D2.columns.tolist()
cols = cols[-1:] + cols[:-1]
D23 = D2[cols]

# Set time as index column
DH = D23.set_index('Time')

In [133]:
# Select the dates:
# Hour resolution data
DH1 = DH.loc['2015-12-01 00:00':'2016-03-01 23:00']['Temperature']
DH2 = DH.loc['2016-06-01 00:00':'2016-09-01 23:00']['Temperature']

## Question 2

### 1) Winter data set:

In [151]:
plt.figure(figsize = (15, 6))
ax1 = DH1.plot(label = 'Temperature')
plt.title('Temperature data from UVic Sci from 1 Dec 2015 to 1 Mar 2016', fontsize=16)
plt.xlabel('Time (hr)', fontsize=14)
plt.ylabel('Temperature ($^\circ C$)', fontsize=14)
ax1.xaxis.set_major_locator(mp.dates.DayLocator(bymonthday = (1, 8, 15, 22)))
plt.show()

<Figure size 1080x432 with 1 Axes>

#### Correlation

In [142]:
correlate(DH1, DH1)

array([ 42.9226,  82.1312, 122.9707, ..., 122.9707,  82.1312,  42.9226])

### 2) Summer data set:

In [150]:
plt.figure(figsize = (15, 6))
ax2 = DH2.plot(label = 'Temperature')
plt.title('Temperature data from UVic Sci from 1 Jun 2016 to 1 Sep 2016',  fontsize=16)
plt.xlabel('Time (hr)', fontsize=14)
plt.ylabel('Temperature ($^\circ C$)', fontsize=14)
ax2.xaxis.set_major_locator(mp.dates.DayLocator(bymonthday = (1, 8, 15, 22)))
plt.show()

<Figure size 1080x432 with 1 Axes>